# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [01:53<00:00, 22.77s/it]


In [6]:
len(deals)

50

In [7]:
deals[44].describe()

"Title: Harbor Freight Tools Scratch and Dent Sale: Up to 45% off + free shipping\nDetails: Save up to 45% off a range of tool storage chests and stainless steel worktops. We've pictured the Icon 19 In. Professional End Cabinet for $629 ($270 off) Stock is limited. Items ship for free, although you're responsible for un-palletizing and setup. You need to call the number on the sale page to place an order for these items. Shop Now at Harbor Freight Tools\nFeatures: \nURL: https://www.dealnews.com/Harbor-Freight-Tools-Scratch-and-Dent-Sale-Up-to-45-off-free-shipping/21760435.html?iref=rss-c196"

As we can see, the above description that has been scraped does not really include a price. This is where we'll use an LLM to use the scraped output to then fetch the price in the form of a "Structured Output"

In [8]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [9]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [10]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung Galaxy Ring: $100 off, or up to $150 off w/ trade-in + free shipping
Details: Trade in an eligible smartwatch or ring and get up to $150 off the new Galaxy Ring. Don't have a trade? You still get $100 off Samsung's sleek, minimalist take on wearable health tech. With the full discount, your final price could be as low as $250. It allows you to upgrade your f

In [12]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection # pass the DealSelection class to parse the response correctly
    )
    result = completion.choices[0].message.parsed
    return result

In [13]:
result = get_recommendations()

In [14]:
len(result.deals)

5

In [15]:
result.deals[1]

Deal(product_description='The EcoFlow Delta 2 is a powerful portable power station equipped with a 1,024Wh capacity, making it perfect for outdoor adventures or emergencies. It features multiple output options including AC, USB, and USB-C ports, ensuring you can charge various devices at once. The EcoFlow Delta 2 supports solar charging (solar panel not included), giving you the flexibility to harness renewable energy when needed. This compact and robust power station ensures you stay powered up wherever you are.', price=490.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-Wh-Portable-Power-Station/473744.html?iref=rss-c142')

In [16]:
from agents.scanner_agent import ScannerAgent

In [17]:
agent = ScannerAgent()
result = agent.scan()

In [18]:
result

DealSelection(deals=[Deal(product_description="The EcoFlow Delta 2 is a powerful portable power station boasting a capacity of 1,024Wh to keep your devices charged on the go. It features multiple charging options including AC and solar inputs, making it versatile for various situations. The unit comes equipped with four USB ports and two USB-C ports, ensuring you can power multiple devices simultaneously. With its compact design, it's ideal for camping trips or emergency backup power, allowing you to stay connected no matter where you are.", price=490.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-Wh-Portable-Power-Station/473744.html?iref=rss-c142'), Deal(product_description='The Blink Outdoor 4 5-Camera System is designed for robust home security with high-definition day and night vision capabilities. It includes five cameras paired with a Sync Module Core, enabling seamless connectivity and control through your smartphone. With a two-year battery life, enh